# Terra Cluster

## Context
In this proyect, we aim to classify land using advanced unsupervised machine learning algorithms facilitated by Google Earth Engine (GEE). Our focus lies on the southern region of Vancouver Island, a critical area where ancient old-growth forests face imminent threats from logging industries failing to comply with logging restrictions. 🌲🌿

## Timeframe
Our analysis spans from January 1, 2019, to January 1, 2023, a timeframe carefully selected to encompass the before, during, and after of the prominent [Fairy Creek old-growth logging protests and blockades](https://en.wikipedia.org/wiki/Fairy_Creek_old-growth_logging_protests) ⏳

## Methods
Our methodology involves a multi-step process:
1. Creating a composite image by calculating the median of all available scenes.
2. Employing image clustering techniques using the `ee.Clusterer.wekaKMeans` algorithm.
3. Visualizing results using geemap.
4. Validation procedures (currently under development).
5. Post-processing tasks in QGIS, including geometry simplification and filtering. 🗺️🌐

## Future Work
Looking ahead, we have exciting plans:
1. Conducting a comprehensive time series analysis to assess the rate of logging over time. This analysis will involve tracking changes in the forested area over different periods, shedding light on the ecological impact.
2. Incorporating Digital Elevation Models (DEMs), NDVI, and other geospatial attributes to enhance the depth and precision of our analysis. 📈🌎

Stay tuned for more updates on this journey! 🌟


In [14]:
import geemap
import ee

## Data Collection and Filtering

In [15]:
# Initialize the Earth Engine library.
ee.Initialize()

# Setting coords for southern Vancouver Island
coords = ee.Geometry.Polygon([
          [
            [
              -126.39491254951932,
              48.34139149028323
            ],
            [
              -123.17356426424683,
              48.34139149028323
            ],
            [
              -123.17356426424683,
              49.445441268636614
            ],
            [
              -126.39491254951932,
              49.445441268636614
            ],
            [
              -126.39491254951932,
              48.34139149028323
            ]
          ]
      ])

collection = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate('2019-01-01', '2023-01-01')
    .filterBounds(coords)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))
)


In [16]:
collection.size()

In [17]:
collection.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1830, 1830],
   'crs': 'EPSG:32609',
   'crs_transform': [60, 0, 600000, 0, -60, 5400000]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32609',
   'crs_transform': [10, 0, 600000, 0, -10, 5400000]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32609',
   'crs_transform': [10, 0, 600000, 0, -10, 5400000]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32609',
   'crs_transform': [10, 0, 600000, 0, -10, 5400000]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min':

In [18]:
# Select one band and get its projection
projection = collection.first().select('B2').projection()

# Get the nominal scale
scale = projection.nominalScale().getInfo()

print("Image scale:", scale)

Image scale: 10


## Timelapse

In [21]:
Map = geemap.Map(zoom=4)
Map.add_basemap('Esri.WorldImagery')
Map.setCenter(-124.34487, 48.61938, 8)
Map

Map(center=[48.61938, -124.34487], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

In [20]:
roi = Map.user_roi

timelapse = geemap.landsat_timelapse(
    roi,
    out_gif='landsat.gif',
    start_year=1984,
    end_year=2022,
    start_date='01-01',
    end_date='12-31',
    bands=['SWIR1', 'NIR', 'Red'],
    frames_per_second=5,
    title='Landsat Timelapse',
    progress_bar_color='blue',
    mp4=True,
)
geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: /Users/karlamuller/code/terra_cluster/landsat.gif
ffmpeg is not installed on your computer.
ffmpeg is not installed on your computer.


Output()

## Adding median image layer

In [8]:
# Band Choice: Some bands are more sensitive to certain features. For example, NIR (Near Infrared) is great for vegetation.
# We will be focusing on Blue, Green, Red, and NIR
selection_bands = ['B2', 'B3', 'B4', 'B8']

filtered_collection = collection.select(selection_bands)

for band in selection_bands:
    # Select one band and get its projection
    filtered_projection = filtered_collection.first().select(band).projection()
    
    # Get the nominal scale
    filtered_scale = filtered_projection.nominalScale().getInfo()
    
    print(f"Image Collection {band} scale: {filtered_scale}")

Image Collection B2 scale: 10
Image Collection B3 scale: 10
Image Collection B4 scale: 10
Image Collection B8 scale: 10


In [9]:
# For land cover classification, we often work with a simple "composite" image rather than numerous scenes
# The median of our collection will reduce noise and give a "typical" representation of the area across all images
# Karla: please keep the original projection and scale when creating the median image with `.setDefaultProjection(crs=filtered_projection, scale=filtered_scale))`
image = filtered_collection.median()
image.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 65535},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 65535},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 65535},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B8',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 65535},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

## Increasing scale 

In [10]:
# # Defining the scale factor
# scale_factor = 5

# # Getting the current image resolution
# original_resolution = image.projection().nominalScale()

# # Calculating the new resolution
# new_resolution = original_resolution.multiply(scale_factor)

# # Upscaling the image using reduceResolution
# upscaled_image = image.reduceResolution(
#     reducer=ee.Reducer.mean(),  
#     maxPixels=60000,  # Reduced 'maxPixels' value to stay below the limit
# ).reproject(crs=image.projection(), scale=new_resolution)

# print("Original Image Resolution:", original_resolution.getInfo())
# print("Upscaled Image Resolution:", new_resolution.getInfo())

## Land Cover Unsupervised Classification

In [11]:
# Sample some points from the image
sample_coords = ee.Geometry.Polygon([
    [
        [-115.1554, 49.4247],
        [-114.9554, 49.4247],
        [-114.9554, 49.6247],
        [-115.1554, 49.6247],
        [-115.1554, 49.4247]
    ]
])


sample = image.sample(region=coords, scale=30, numPixels=5000)

# Train the clusterer
trained_clusterer = ee.Clusterer.wekaKMeans(7).train(sample)

# Cluster the image
clustered_image = image.cluster(trained_clusterer)

## Visualization

In [12]:
# Median composite 
image_params = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
}

Map = geemap.Map(zoom=4)
Map.add_basemap('Esri.WorldImagery')
Map.setCenter(-124.34487, 48.61938, 8)
Map.addLayer(image, image_params, 'Sentinel-2')

# Create a dict for legend labels and their corresponding colors
legend_info = {
    'Mixed Forest': '#FF0000',  # Red 
    'Snow': '#FFA500',  # Orange 
    'Exposed Land': '#FFC0CB',  # Pink
    'Snow': '#00FF00',  # Green 
    'Coniferous Forest': '#0000FF',  # Blue 
    'Developed/Exposed Land': '#EE82EE',  # Violet 
    'Water': '#FFFF00'  # Yellow 
}

# Create visualization parameters
# TODO: why palette keeps changing on me?!
cluster_params = {'min': 0, 'max': 6, 'palette': list(legend_info.values())}

# Add layers and legend to the map
Map.addLayer(clustered_image, cluster_params, 'Clusters')
Map.add_legend(keys=list(legend_info.keys()), colors=list(legend_info.values()), position='bottomright')

In [13]:
Map

Map(center=[48.61938, -124.34487], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

## Convert Clustered Image to Vector

In [52]:
clustered_vector = clustered_image.reduceToVectors(
    geometry=coords, 
    scale=30, 
    maxPixels=1e13,  
    bestEffort=True 
)

In [ ]:
task = ee.batch.Export.table.toDrive(
    collection=clustered_vector,
    description='clustered_svi_vector',
    folder='terra_cluster',
    fileFormat='SHP'
)
task.start()

# Polygon Reduction Strategies

Oh no, we have an abundance of polygons! It looks like a sea of Tetris blocks! Here are some ideas to reduce the number of polygons:

1. **Change Cluster Number**: Consider reducing the cluster count to generalize the areas more. ✅
   - Reducing the number of clusters was tested, but it failed a visual inspection of forested area classifications.
   - Current value: 7

2. **Adjust Scale**: Increasing the scale will result in coarser, larger blocks, potentially reducing the number of polygons. ✅
   - Adjusting the scale proved effective in reducing the number of polygons.

3. **Post-processing**: After generating the polygons, try employing geometry simplification techniques to merge adjacent polygons with the same cluster label. (IN QGIS)

4. **Filter by Size**: You can remove very small polygons that may not be significant for your analysis. (IN QGIS)

5. **Adding Additional Attributes**: Consider incorporating DEMs, NDVI, temperature or other relevant geospatial attributes for enhanced analysis. 📊🌡️
